In [1]:
# import h2o package and specific estimator 
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')

In [2]:
h2o.init(strict_version_check=False) # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.201-b09, mixed mode)
  Starting server from C:\Users\Ashu Kapil\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ASHUKA~1\AppData\Local\Temp\tmp_gwsv7j5
  JVM stdout: C:\Users\ASHUKA~1\AppData\Local\Temp\tmp_gwsv7j5\h2o_Ashu_Kapil_started_from_python.out
  JVM stderr: C:\Users\ASHUKA~1\AppData\Local\Temp\tmp_gwsv7j5\h2o_Ashu_Kapil_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_Ashu_Kapil_fv4s43
H2O cluster total nodes:,1
H2O cluster free memory:,3.523 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
#importing data to the server
hp = h2o.import_file(path="hour.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
#Displaying the head
hp.head()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01 00:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01 00:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01 00:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01 00:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01 00:00:00,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01 00:00:00,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01 00:00:00,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01 00:00:00,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01 00:00:00,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14


In [5]:
hp.describe()

Rows:17379
Cols:17




,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
type,int,time,int,int,int,int,int,int,int,int,real,real,real,real,int,int,int
mins,1.0,1293840000000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,0.0,1.0
mean,8690.0,1325477314552.046,2.5016399102364923,0.5025605615973301,6.537775476149376,11.546751826917545,0.028770355026181024,3.003682605443351,0.6827205247712756,1.4252833879970077,0.4969871684216583,0.4757751021347602,0.6272288394038783,0.1900976063064618,35.67621842453534,153.78686920996606,189.46308763450145
maxs,17379.0,1356912000000.0,4.0,1.0,12.0,23.0,1.0,6.0,1.0,4.0,1.0,1.0,1.0,0.8507,367.0,886.0,977.0
sigma,5017.029499614288,18150225217.779423,1.1069181394480763,0.5000078290910197,3.438775713750165,6.914405095264493,0.16716527638437123,2.0057714561109883,0.4654306335238829,0.6393568777542534,0.1925561212497219,0.1718502156353594,0.19292983406291508,0.1223402285727905,49.30503038705308,151.35728591258314,181.38759909186476
zeros,0,0,0,8645,0,726,16879,2502,5514,0,0,2,22,2180,1581,24,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,2011-01-01 00:00:00,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01 00:00:00,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,8.0,32.0,40.0
2,3.0,2011-01-01 00:00:00,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,5.0,27.0,32.0


In [6]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [7]:
all_variables=None

In [8]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1000
run_id='SOME_ID_20180617_221529' # Just some arbitrary ID
server_path='/Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/RunTime_1000Seconds'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [9]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'/Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/RunTime_1000Seconds\\hour.csv'

In [10]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [11]:
# determine column types
ints, reals, enums = [], [], []
for key, val in hp.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [12]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = hp[reals].scale()
    hp[ints] = hp[ints].scale()

In [13]:
hp[y].levels()

[]

In [14]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1000


int

## Cross-validate rather than take a test training split with 1000 seconds

In [15]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1000 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [16]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DeepLearning_1_AutoML_20190416_164439,9.17038,3.02826,9.17038,2.24116,nan
GBM_2_AutoML_20190416_164439,12.8769,3.58844,12.8769,2.09538,0.0348002
GBM_1_AutoML_20190416_164439,18.0864,4.25281,18.0864,2.6474,0.0656178
GBM_grid_1_AutoML_20190416_164439_model_2,19.6573,4.43366,19.6573,2.71491,0.092278
GLM_grid_1_AutoML_20190416_164439_model_1,38.387,6.19573,38.387,4.45197,nan
XRT_1_AutoML_20190416_164439,44.3588,6.66024,44.3588,3.6048,0.0777871
GBM_grid_1_AutoML_20190416_164439_model_5,45.6403,6.75576,45.6403,4.15727,0.110496
GBM_3_AutoML_20190416_164439,46.2606,6.80151,46.2606,4.01548,nan
DRF_1_AutoML_20190416_164439,46.6841,6.83258,46.6841,3.69712,0.0777093
GBM_5_AutoML_20190416_164439,47.0687,6.86066,47.0687,3.9562,nan


In [17]:
aml.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_20190416_164439


ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 3.6521750869518255
RMSE: 1.9110664789461997
MAE: 1.3038189883865567
RMSLE: NaN
Mean Residual Deviance: 3.6521750869518255

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 9.170377209704913
RMSE: 3.02826306811428
MAE: 2.2411642127330214
RMSLE: NaN
Mean Residual Deviance: 9.170377209704913
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,2.2411337,0.4935897,2.302596,3.4547353,2.317202,1.4221529,1.7089815
mean_residual_deviance,9.170138,3.2957633,8.062223,17.675018,10.053273,5.0380034,5.022173
mse,9.170138,3.2957633,8.062223,17.675018,10.053273,5.0380034,5.022173
r2,0.9997217,0.0000991,0.9997553,0.9994669,0.9996927,0.9998472,0.9998465
residual_deviance,9.170138,3.2957633,8.062223,17.675018,10.053273,5.0380034,5.022173
rmse,2.9399664,0.5131937,2.8394055,4.204167,3.1706898,2.2445498,2.2410204
rmsle,0.0976188,0.0022646,NaN,NaN,0.1001507,0.0950869,NaN


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2
,2019-04-16 16:47:30,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-04-16 16:47:30,4.464 sec,149603 obs/sec,1.0416019,1,18102.0,20.6402278,426.0190020,15.9522321,0.9869935
,2019-04-16 16:47:31,5.753 sec,142957 obs/sec,11.4257437,11,198568.0,1.9110665,3.6521751,1.3038190,0.9998885


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,1.0,1.0,0.2776948
holiday,0.3651319,0.3651319,0.1013952
casual,0.3641420,0.3641420,0.1011204
dteday,0.2782135,0.2782135,0.0772584
instant,0.2717896,0.2717896,0.0754746
atemp,0.2278138,0.2278138,0.0632627
yr,0.1882673,0.1882673,0.0522809
mnth,0.1722641,0.1722641,0.0478368
temp,0.1613016,0.1613016,0.0447926
season,0.1297598,0.1297598,0.0360336


In [18]:
aml.leader.algo

'deeplearning'

## Getting Models

In [19]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [20]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DeepLearning_1_AutoML_20190416_164439',
   'type': 'Key<Model>',
   'URL': '/3/Models/DeepLearning_1_AutoML_20190416_164439'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'default': None, 'actual': None},
 'response_column': {'de

In [21]:
hy_parameter = mod_best.params
n=run_id+'_hy_parameter_XGBoost_1_AutoML_20190415_132427.json'
dict_to_json(hy_parameter,n)

In [22]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_20190416_164439


ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 3.6521750869518255
RMSE: 1.9110664789461997
MAE: 1.3038189883865567
RMSLE: NaN
Mean Residual Deviance: 3.6521750869518255

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 9.170377209704913
RMSE: 3.02826306811428
MAE: 2.2411642127330214
RMSLE: NaN
Mean Residual Deviance: 9.170377209704913
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,2.2411337,0.4935897,2.302596,3.4547353,2.317202,1.4221529,1.7089815
mean_residual_deviance,9.170138,3.2957633,8.062223,17.675018,10.053273,5.0380034,5.022173
mse,9.170138,3.2957633,8.062223,17.675018,10.053273,5.0380034,5.022173
r2,0.9997217,0.0000991,0.9997553,0.9994669,0.9996927,0.9998472,0.9998465
residual_deviance,9.170138,3.2957633,8.062223,17.675018,10.053273,5.0380034,5.022173
rmse,2.9399664,0.5131937,2.8394055,4.204167,3.1706898,2.2445498,2.2410204
rmsle,0.0976188,0.0022646,NaN,NaN,0.1001507,0.0950869,NaN


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2
,2019-04-16 16:47:30,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-04-16 16:47:30,4.464 sec,149603 obs/sec,1.0416019,1,18102.0,20.6402278,426.0190020,15.9522321,0.9869935
,2019-04-16 16:47:31,5.753 sec,142957 obs/sec,11.4257437,11,198568.0,1.9110665,3.6521751,1.3038190,0.9998885


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,1.0,1.0,0.2776948
holiday,0.3651319,0.3651319,0.1013952
casual,0.3641420,0.3641420,0.1011204
dteday,0.2782135,0.2782135,0.0772584
instant,0.2717896,0.2717896,0.0754746
atemp,0.2278138,0.2278138,0.0632627
yr,0.1882673,0.1882673,0.0522809
mnth,0.1722641,0.1722641,0.0478368
temp,0.1613016,0.1613016,0.0447926
season,0.1297598,0.1297598,0.0360336


<bound method ModelBase.coef_norm of >


### Generating all the parameters for the models for json file in the below code

In [23]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[5])

In [24]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XRT_1_AutoML_20190416_164439',
   'type': 'Key<Model>',
   'URL': '/3/Models/XRT_1_AutoML_20190416_164439'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 0},
 'fold_assignment': {'default': 'AUTO',

In [25]:
hy_parameter = mod_best.params
n=run_id+'_hy_parameter_GLM_grid_1_AutoML_20190415_132427_model_1.json'
dict_to_json(hy_parameter,n)

### Parameters for StackedEnsemble_AllModels_AutoML_20190408_134347

In [26]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[4])

In [27]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GLM_grid_1_AutoML_20190416_164439_model_1',
   'type': 'Key<Model>',
   'URL': '/3/Models/GLM_grid_1_AutoML_20190416_164439_model_1'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'seed': {'default': -1, 'actual': -1669996477772821136},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_co

In [28]:
hy_parameter = mod_best.params
n=run_id+'_hy_parameter_GLM_grid_1_AutoML_20190415_132427_model_1.json'
dict_to_json(hy_parameter,n)

### Parameters for XGBoost_3_AutoML_20190415_140606

In [29]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[2])

In [30]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_1_AutoML_20190416_164439',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_1_AutoML_20190416_164439'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

### Parameters for GLM_grid_1_AutoML_20190415_140606_model_1

In [31]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[9])

In [32]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_5_AutoML_20190416_164439',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_5_AutoML_20190416_164439'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO',

In [33]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_5_AutoML_20190416_164439


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 24.523935523009555
RMSE: 4.952164731004973
MAE: 2.833811991331722
RMSLE: NaN
Mean Residual Deviance: 24.523935523009555

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 47.06865585076463
RMSE: 6.860660015681044
MAE: 3.9561969592340644
RMSLE: NaN
Mean Residual Deviance: 47.06865585076463
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,3.9561958,0.0587636,4.057553,4.044572,3.84221,3.9011989,3.9354455
mean_residual_deviance,47.068546,1.3762751,48.166687,48.89247,44.29227,48.824635,45.166668
mse,47.068546,1.3762751,48.166687,48.89247,44.29227,48.824635,45.166668
r2,0.9985696,0.0000373,0.9985378,0.9985253,0.9986461,0.9985188,0.9986197
residual_deviance,47.068546,1.3762751,48.166687,48.89247,44.29227,48.824635,45.166668
rmse,6.859172,0.1007668,6.9402223,6.9923153,6.655244,6.9874625,6.720615
rmsle,0.0913594,0.0006121,NaN,0.0920437,NaN,0.0906750,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-16 16:47:20,22.183 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-16 16:47:20,22.298 sec,5.0,110.7322781,86.3277563,12261.6374103
,2019-04-16 16:47:20,22.409 sec,10.0,71.9314240,55.2145545,5174.1297655
,2019-04-16 16:47:20,22.523 sec,15.0,44.8787055,34.1408197,2014.0982104
,2019-04-16 16:47:20,22.653 sec,20.0,28.7696894,21.4192446,827.6950288
---,---,---,---,---,---,---
,2019-04-16 16:47:24,26.615 sec,200.0,5.1198121,2.9319914,26.2124763
,2019-04-16 16:47:24,26.717 sec,205.0,5.0781225,2.9085056,25.7873281
,2019-04-16 16:47:24,26.838 sec,210.0,5.0386905,2.8872145,25.3884018
,2019-04-16 16:47:24,26.951 sec,215.0,4.9882502,2.8544063,24.8826402



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,1706243712.0000000,1.0,0.6275278
hr,497373120.0000000,0.2915018,0.1829255
casual,296937472.0000000,0.1740299,0.1092086
dteday,63506368.0000000,0.0372200,0.0233566
atemp,37039932.0000000,0.0217085,0.0136227
temp,36623080.0000000,0.0214642,0.0134694
instant,33953964.0000000,0.0198998,0.0124877
workingday,25527070.0000000,0.0149610,0.0093884
weekday,10738259.0000000,0.0062935,0.0039494
hum,5102961.0,0.0029908,0.0018768


<bound method ModelBase.coef_norm of >


### Parameters for GLM_grid_1_AutoML_20190415_132427_model_1

In [35]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[18])

In [36]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_1_AutoML_20190416_164439_model_3


ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 458.26558268923714
RMSE: 21.407138591816448
MAE: 16.857456851245278
RMSLE: NaN
Mean Residual Deviance: 458.26558268923714

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 731.6578490700517
RMSE: 27.049174646743875
MAE: 21.008085012130703
RMSLE: NaN
Mean Residual Deviance: 731.6578490700517
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,21.008194,1.8056083,16.185268,21.137619,21.453772,23.36007,22.904242
mean_residual_deviance,731.66675,122.9544750,433.93967,679.6674,737.3275,921.16486,886.2342
mse,731.66675,122.9544750,433.93967,679.6674,737.3275,921.16486,886.2342
r2,0.977752,0.0037596,0.9868274,0.9794998,0.9774617,0.9720547,0.9729164
residual_deviance,731.66675,122.9544750,433.93967,679.6674,737.3275,921.16486,886.2342
rmse,26.835161,2.4021676,20.831219,26.070433,27.153776,30.350698,29.769686
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2
,2019-04-16 16:55:25,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-04-16 16:55:25,1 min 57.924 sec,18239 obs/sec,0.5740837,1,9977.0,16.7589480,280.8623393,13.5016535,0.9916483
,2019-04-16 16:55:30,2 min 3.063 sec,17807 obs/sec,5.7535531,10,99991.0,21.4071386,458.2655827,16.8574569,0.9863731


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,1.0,1.0,0.1549770
casual,0.5959838,0.5959838,0.0923638
hr,0.5738292,0.5738292,0.0889303
hum,0.3609922,0.3609922,0.0559455
atemp,0.3554605,0.3554605,0.0550882
dteday,0.3514147,0.3514147,0.0544612
temp,0.3478805,0.3478805,0.0539135
holiday,0.3419848,0.3419848,0.0529998
season,0.3361935,0.3361935,0.0521023
workingday,0.3357315,0.3357315,0.0520307


<bound method ModelBase.coef_norm of >
